In [1]:
import random
import itertools
import collections
import subprocess
import json

def solve(args,clingo_exe='clingo'):
    """Run clingo with the provided argument list and return the parsed JSON result."""

    print_args = [clingo_exe] + list(args) + [' | tr [:space:] \\\\n | sort ']
    args = [clingo_exe, '--outf=2'] + args # + ["--sign-def=rnd","--seed="+str(random.randint(0,1<<30))] #No randomness here
    #print(' '.join(args))
    with subprocess.Popen(
        ' '.join(args),
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        shell=True
    ) as clingo:
        outb, err = clingo.communicate()
    #if err:
    #    print(err)
    out = outb.decode("utf-8")
    if len(out) == 0:
        print(f'Command "{" ".join(args)}" failed.')
    with open('dump.lp', 'w') as outfile:
        result = json.loads(out)
        witness = result['Call'][0]['Witnesses'][-1]['Value']
        for atom in sorted(witness):
            outfile.write(atom + '.\n')
    return parse_json_result(out) ,   

def parse_terms(arguments):
    terms = []
    while len(arguments) > 0:
        l_paren = arguments.find('(')
        r_paren = arguments.find(')')
        comma = arguments.find(',')
        if l_paren < 0:
            l_paren = len(arguments) - 1
        if r_paren < 0:
            r_paren = len(arguments) - 1
        if comma < 0:
            comma = len(arguments) - 1
        next = min(l_paren, r_paren, comma)
        next_c = arguments[next]
        if next_c == '(':

            pred = arguments[:next]
            sub_terms, arguments = parse_terms(arguments[next + 1:])
            terms.append({'predicate': pred, 'terms': sub_terms})
        elif next_c == ')':
            pred = arguments[:next]
            if pred != '':
                terms.append({'predicate': arguments[:next]})
            arguments = arguments[next + 1:]
            return terms, arguments
        elif next_c == ',':
            pred = arguments[:next]
            if pred != '':
                terms.append({'predicate': arguments[:next]})
            arguments = arguments[next + 1:]
        else:
            terms.append({'predicate': arguments})
            arguments = ''
    return terms, ''


def parse_json_result(out):
    """Parse the provided JSON text and extract a dict
    representing the predicates described in the first solver result."""
    result = json.loads(out)
    assert len(result['Call']) > 0
    assert len(result['Call'][0]['Witnesses']) > 0
    all_preds = []
    ids = list(range(len(result['Call'][0]['Witnesses'])))
    random.shuffle(ids)
    for id in ids[:]:
        witness = result['Call'][0]['Witnesses'][id]['Value']

        class identitydefaultdict(collections.defaultdict):
            def __missing__(self, key):
                return key

        preds = collections.defaultdict(list)
        env = identitydefaultdict()

        for atom in witness:
            parsed, dummy = parse_terms(atom)
            preds[parsed[0]['predicate']].append(parsed)
        all_preds.append(preds)
    return all_preds


In [4]:
%%time 
import numpy as np
rules = 'no_age_sampling'
characters = 200
sample_ages = False
if sample_ages:
    ages = np.random.randn(characters,1)*35/2 + 35
    ages[ages<=1] += 35
    ages[ages>=70] -= 35
    bit = True


    with open('ages.lp','w') as outfile:
        if bit:
            rules = 'no_age_sampling'
            for charID, age in enumerate(ages):
                age = int(age)
                bit = 0
                while age > 0:
                    if age % 2 == 1:
                        outfile.write(f'age_bits(character({charID+1}),{bit}).')
                    age = age // 2
                    bit += 1
        else:
            rules = 'no_age_num'
            for charID, age in enumerate(ages):
                age = int(age)
                outfile.write(f'age(character({charID+1}),{age}).')
                print(age)
print(f'clingo {rules} ages.lp -n 1 -t 8 -c characters={characters}')

old_way = False
if old_way:
    results = !clingo {rules} ages.lp  -n 1 -t 8 -c characters={characters}
    age_bits = {}
    for result in results:
        for thing in sorted(result.split(' ')):
            print(thing)
            if True:
                if 'age_bits' in thing[:9]:
                    thing = thing[18:]
                    char_id = thing.split(')')[0][1:]

                    age_bit = thing.split(',')[1][:-1]
                    age_bits[char_id] = age_bits.get(char_id,[]) + [int(age_bit)]

    for char in age_bits:
        age = 0
        for bit in age_bits[char]:
            age += 2**bit
        print(char,age)
else:
    solution = solve([rules, 'ages.lp','-n','1','-t','8','-c',f'characters={characters}'])
    !clingo dump.lp -t 8 residence_and_jobs.lp -c characters={characters}

clingo no_age_sampling ages.lp -n 1 -t 8 -c characters=200
clingo version 5.4.0
Reading from dump.lp ...
Solving...
Answer: 1
residence_type(mansion) residence_type(tenement) residence_type(house) is(character(1),extrovert) is(character(1),gregarious) is(character(1),rich) is(character(1),selfish) is(character(10),cantankerous) is(character(10),introvert) is(character(10),middleclass) is(character(10),selfish) is(character(100),cantankerous) is(character(100),introvert) is(character(100),middleclass) is(character(100),selfish) is(character(101),cantankerous) is(character(101),extrovert) is(character(101),middleclass) is(character(101),selfish) is(character(102),extrovert) is(character(102),generous) is(character(102),gregarious) is(character(102),middleclass) is(character(103),cantankerous) is(character(103),generous) is(character(103),introvert) is(character(103),rich) is(character(104),cantankerous) is(character(104),introvert) is(character(104),manic_depressive) is(character(104),mi

In [65]:
!clingo dump.lp -t 8 residence_and_jobs.lp -c characters={characters} 

clingo version 5.4.0
Reading from dump.lp ...
Solving...
Answer: 1
character(1) character(10) character(100) character(101) character(102) character(103) character(104) character(105) character(106) character(107) character(108) character(109) character(11) character(110) character(111) character(112) character(113) character(114) character(115) character(116) character(117) character(118) character(119) character(12) character(120) character(121) character(122) character(123) character(124) character(125) character(126) character(127) character(128) character(129) character(13) character(130) character(131) character(132) character(133) character(134) character(135) character(136) character(137) character(138) character(139) character(14) character(140) character(141) character(142) character(143) character(144) character(145) character(146) character(147) character(148) character(149) character(15) character(150) character(151) character(152) character(153) character(154) character(1

In [ ]:
%%time
rules = !gringo condensed.lp --text 
print(len(rules))

In [ ]:
%%time 
!clasp grounded

In [ ]:
!clingo processed.lp init_test_stage_2.lp

In [ ]:
#W: 8.89 
rules = !clingo init_test.lp --text
print(len(rules))
#45650 without spouses living in same house

In [ ]:
!clingo small_test --text

In [ ]:
%%time
results = !clingo small_test 

age_bits = {}
for result in results:
    for thing in sorted(result.split(' ')):
        if 'mean' in thing:
            print(thing)
        if True:
            if 'age_bits' in thing[:9]:
                thing = thing[18:]
                char_id = thing.split(')')[0][1:]

                age_bit = thing.split(',')[1][:-1]
                age_bits[char_id] = age_bits.get(char_id,[]) + [int(age_bit)]

for char in age_bits:
    age = 0
    for bit in age_bits[char]:
        age += 2**bit
    print(char,age)

In [ ]:
text_rules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
plt.plot([i%10for i in range(100)],[i%7 for i in range(100)],'x')
plt.show()

In [ ]:
%%time
results = !clingo small_test 
print(len(results))
#Full Fat = .139
#Trimmed = 0.01
#Double Trimmed

In [ ]:
print(results)

In [ ]:
import matplotlib.pyplot as plt
for j in range(2,3):
    for f in range(4,9+1):
        data = []
        for c in range(1,100+1):
            data.append(c % (j*f))
        plt.plot(list(range(1,100+1)),data,'x')
        plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
symbols = ['.','s','o','x','+']
colors = ['k','r','g','b','c']
starting_point = 2

counts = {}
for j in range(2,3):
    for f in range(10+starting_point,starting_point,-1):
        data = []
        for c in range(1,100+1):
            for o in range(1,100+1):
                if c % (j*f) == 0 and o % (j*f) == 0:
                    data.append((c,o))
                    counts[(c,o)] = counts.get((c,o),0)+1
        data = np.array(data)            
        plt.plot(data[:,0],data[:,1],symbols[(j+f)%len(symbols)]+colors[(j+f)%len(colors)])
    plt.show()
for p in counts:
    if counts[p] > 1:
        print(p,counts[p])

In [ ]:
print(data[:,1])

In [ ]:
7/2*100/(10*7)
